In [1]:
import gym
env = gym.make("LunarLanderContinuous-v2")

[2022-09-20 00:23:09,470] Making new env: LunarLanderContinuous-v2
/usr/local/lib/python3.8/dist-packages/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [2]:
import numpy as np

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [4]:
device = torch.device("cpu")


In [5]:
env.reset()

array([ 0.00325508,  0.93802462,  0.32968571, -0.172589  , -0.003765  ,
       -0.07467871,  0.        ,  0.        ])

In [6]:
env.step(np.array([0.001, 0.001]))

(array([ 0.00642653,  0.93533574,  0.32128093, -0.17927957, -0.00786309,
        -0.08196877,  0.        ,  0.        ]),
 0.12848183525954585,
 False,
 {})

In [7]:
class Policy(nn.Module):
    def __init__(self, s_size=4, fc1_size=150, fc2_size=120, a_size=2):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, fc1_size)
        self.fc2 = nn.Linear(fc1_size, fc2_size)
        self.mu = nn.Linear(fc2_size, a_size)
        self.std = nn.Linear(fc2_size, a_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        mu = self.mu(x)
        std = F.relu(self.std(x))
        return mu, std
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        mu, std = self.forward(state)
        print(f'DEBUG: std {std}')
        action = torch.normal(mu, std)
        return action
        # TODO (sezan92):
        # get the probability from mu and std
        
        # return action.item(), m.log_prob(action)

In [8]:
model = Policy(s_size=8)

In [9]:
state = env.reset()

In [10]:
state

array([ 0.00612793,  0.93864432,  0.62068253, -0.13128264, -0.00709402,
       -0.14059384,  0.        ,  0.        ])

In [11]:
action = model.act(state)

DEBUG: std tensor([[0.0000, 0.0500]], grad_fn=<ReluBackward0>)


In [12]:
action

tensor([[ 0.1167, -0.0413]], grad_fn=<NormalBackward3>)

In [13]:
# TODO:
# Get the correct and best function for action and log probability for pytorch
# apply
# check if the values are okay.